<a href="https://colab.research.google.com/github/nmhkha/DL_Practice/blob/main/Prac3_Ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login

login() #login hugging face

In [2]:
!pip install transformers

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #subword -> ID

def tokenize(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True) #Padding: Thêm vào cho đủ bằng token [PAD], truncation: cắt bớt phần dư ở cuối nếu dài hơn

dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [22]:
dataset.save_to_disk("/content/imdb_tokenized")


Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
small_train = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = dataset["test"].shuffle(seed=42).select(range(1000))


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [7]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

def compute_metric(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="distilbert_base",
    eval_strategy="epoch",
    push_to_hub=False,

)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metric,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nmhkha123 (nmhkha123-ut-edu-vn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.275800,0.344254,0.889720
2,0.149100,0.283351,0.924920
3,0.067800,0.372458,0.928880


TrainOutput(global_step=9375, training_loss=0.18608028788248698, metrics={'train_runtime': 5049.5038, 'train_samples_per_second': 14.853, 'train_steps_per_second': 1.857, 'total_flos': 9935054899200000.0, 'train_loss': 0.18608028788248698, 'epoch': 3.0})

In [15]:
!pip install torch


In [21]:
import torch
test = "This movie is not good"
input = tokenizer(test, return_tensors="pt", truncation=True)
# Move input to GPU
input = {k: v.to("cuda") for k, v in input.items()}
with torch.no_grad():
  output = model(**input)

predicted_class = torch.argmax(output.logits, dim=1).item()
labels = {0: "Negative", 1: "Positive"}
print(labels[predicted_class])

Negative


In [23]:
trainer.save_model("my_finetuned_model")
tokenizer.save_pretrained("my_finetuned_model")


('my_finetuned_model/tokenizer_config.json',
 'my_finetuned_model/special_tokens_map.json',
 'my_finetuned_model/vocab.txt',
 'my_finetuned_model/added_tokens.json',
 'my_finetuned_model/tokenizer.json')